Helpful Links
 - Reshaping Long Data: https://towardsdatascience.com/3-easy-ways-to-reshape-pandas-dataframe-5b2cbe73d60e
 - Detailed Info on MCR Sheets: https://www.costreportdata.com/worksheet_formats.html

Import Standard Modules and Functions

In [ ]:
import os
from pathlib import Path
from matplotlib.ticker import PercentFormatter
import openpyxl
from datetime import date

# Download ZIP Files
import requests, zipfile
from io import BytesIO

path = os.getcwd()
os.chdir('/Users/CanonPattillo/OneDrive - Porter/Desktop/Stats and DS/Canon Functions')

from canon_functions import *
import canon_functions as cf

os.chdir(path)
print(path)

pd.set_option('display.max_columns', None)

Import Medicare Cost Report Preprocessed Data

In [ ]:
FYyear = input("Which Fiscal Year (YYYY) would you like to process? >>>  ")

In [ ]:
df = pd.read_csv(f'/Users/CanonPattillo/OneDrive - Porter/Desktop/Analysis Central/Sales Support and PlanSystem Intelligence/Version Two/Medicare Cost Reports/Outputs/Unprocessed_{FYyear}_Medicare_Cost_Report.csv'
                , dtype='str')

<font color = 'green'> Isolate those facilities with > 1x MCR for the FY

In [ ]:
# ***** These are the specific facilities with > 1 MCR for the FY
moreThanOneReport = df.groupby(['Provider Number'], as_index=False)['Report Record Number'].nunique()\
.sort_values('Report Record Number', ascending = False)

moreThanOneReport = moreThanOneReport[moreThanOneReport['Report Record Number'] >1]

greThanOne = moreThanOneReport['Provider Number'].unique().tolist()

Column Headers

In [ ]:
# for col in df.columns.tolist():
#     print(f"'{col}' : '{col}',")
    
df = df.rename(columns = {'Report Record Number' : 'Report Record Number',
'Worksheet Identifier' : 'Worksheet Identifier',
'Line Number' : 'Line Number',
'Column Number' : 'Column Number',
'Item Value Number' : 'Value',
'10 FIELD NAME' : '10 Field Name',
'FIELD DESCRIPTION ' : 'Field Description',
'Provider Control Type Code' : 'Provider Control Type Code',
'Provider Number' : 'Provider Number',
'National Provider Identifier' : 'National Provider Identifier',
'Report Status Code' : 'Report Status Code',
'Fiscal Year Begin Date' : 'Fiscal Year Begin Date',
'Fiscal Year End Date' : 'Fiscal Year End Date',
'Process Date' : 'Process Date',
'Initial Report Switch' : 'Initial Report Switch',
'Last Report Switch' : 'Last Report Switch',
'The current transmittal or version number in effect for each sub-system.' : 'The current transmittal or version number in effect for each sub-system.',
'Fiscal Intermediary Number' : 'Fiscal Intermediary Number',
'Automated Desk Review Vendor Code' : 'Automated Desk Review Vendor Code',
'Fiscal Intermediary Create Date' : 'Fiscal Intermediary Create Date',
'Utilization Code' : 'Utilization Code',
'Notice of Program Reimbursement Date' : 'Notice of Program Reimbursement Date',
'Special Indicator' : 'Special Indicator',
'Fiscal Intermediary Receipt Date' : 'Fiscal Intermediary Receipt Date'})

Create Facility Type Crosswalk from:
 - HCRIS_FACILITY_NUMBERING.csv <br>

Code<br>
facilityType = pd.read_csv(f'/Users/CanonPattillo/OneDrive - Porter/Desktop/Analysis Central/Sales Support and PlanSystem Intelligence/Version Two/Medicare Cost Reports/Cost Report Documentation/HCRIS_FACILITY_NUMBERING.csv', dtype='str')

In [ ]:
uniqueCCNs = df['Provider Number'].unique().tolist()

fType = pd.DataFrame(uniqueCCNs, columns =['Provider Number'])

fType['L4'] = fType['Provider Number'].str[-4:]

In [ ]:
# Logic to Identify Hospital Type

fType['Facility Type'] = np.where(fType['L4'] <= '0879', 'Short Term Acute Care', 
                        np.where((fType['L4'] >= '3300') & (fType['L4'] <= '3399'), 'Childrens', 
                        np.where((fType['L4'] >= '1300') & (fType['L4'] <= '1399'), 'Critical Access',
                        np.where((fType['L4'] >= '2000') & (fType['L4'] <= '2299'), 'Long Term Acute Care',
                                 'Other')
                                )
                                )
                        )

In [ ]:
df = df.merge(fType,
             how = 'left',
             left_on = 'Provider Number',
             right_on = 'Provider Number')

Type of Control

In [ ]:
controlDict = {'1' :  'Voluntary Nonprofit-Church',
'2' :  'Voluntary Nonprofit-Other',
'3' :  'Proprietary-Individual',
'4' :  'Proprietary-Corporation',
'5' :  'Proprietary-Partnership',
'6' :  'Proprietary-Other',
'7' :  'Governmental-Federal',
'8' :  'Governmental-City-County',
'9' :  'Governmental-County',
'10' : 'Governmental-State',
'11' : 'Governmental-Hospital District',
'12' : 'Governmental-City',
'13' : 'Governmental-Other'}

In [ ]:
df['Provider Control Type']= df['Provider Control Type Code'].map(controlDict)

Report Status Code

In [ ]:
reportStatusDict = {'1':'As Submitted',
                   '2':'Settled w/o Audit',
                   '3':'Settled with Audit',
                   '4':'Reopened',
                   '5':'Amended'}

In [ ]:
df['Report Status']= df['Report Status Code'].map(reportStatusDict)

Remove Null Columns

In [ ]:
df = df.dropna(axis=1, how='all')

Columns Order

In [ ]:
df = df[['Provider Number', 'Report Record Number', 'Worksheet Identifier', 'Line Number',
       'Column Number', '10 Field Name', 'Field Description', 'Value', 
       'Facility Type', 'Provider Control Type Code', 'Provider Control Type', 'Report Status Code', 'Report Status',
       'Fiscal Year Begin Date', 'Fiscal Year End Date', 'Process Date',
       'Initial Report Switch', 'Last Report Switch',
       'The current transmittal or version number in effect for each sub-system.',
       'Fiscal Intermediary Number', 'Automated Desk Review Vendor Code',
       'Fiscal Intermediary Create Date', 'Utilization Code',
       'Notice of Program Reimbursement Date',
       'Fiscal Intermediary Receipt Date']]

Final Look

In [ ]:
df.sample()

<font color = 'blue'> ### Reduce for Development <font>

# *** PATH 1

In [ ]:
df=df[df['Provider Number'] == '010001']

# *** PATH 2

In [ ]:
# Reduce to Hopkins for Development
hopkins = ['090005',
#            '103300',
#            '210009',
#            '210022',
#            '210029',
#            '210048'
            ]

jh = df[df['Provider Number'].isin(hopkins)].copy().reset_index(drop = True)

Duplicates

In [ ]:
# jh[jh.duplicated(subset=['10 FIELD NAME'])]
# jh[jh.duplicated(subset=['Provider Number', 'Worksheet Identifier', 'Line Number','Column Number'])]
# jh[(jh['Provider Number'] == '090005')&(jh['Worksheet Identifier'] == 'S200001')
#    &(jh['Line Number'] == '02200')&(jh['Column Number'] == '00100')]

<font color = 'blue'> ### Reduce for Development <font>

In [ ]:
jh